Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


# Phase 2 Project

### Importing Data

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#### Formatting Cell

In [ ]:
pd.set_option('display.max_rows', 1000) #change the amount of rows displayed

In [ ]:
df = pd.read_csv('Data/kc_house_data.csv')
df.head()

### Column Names and descriptions for Kings County Data Set
(source: Data/column_names.md)
* **id** - unique identified for a house
* **dateDate** - house was sold
* **pricePrice** -  is prediction target
* **bedroomsNumber** -  of Bedrooms/House
* **bathroomsNumber** -  of bathrooms/bedrooms
* **sqft_livingsquare** -  footage of the home
* **sqft_lotsquare** -  footage of the lot
* **floorsTotal** -  floors (levels) in house
* **waterfront** - House which has a view to a waterfront
* **view** - Has been viewed
* **condition** - How good the condition is ( Overall )
* **grade** - overall grade given to the housing unit, based on King County grading system
* **sqft_above** - square footage of house apart from basement
* **sqft_basement** - square footage of the basement
* **yr_built** - Built Year
* **yr_renovated** - Year when house was renovated
* **zipcode** - zip
* **lat** - Latitude coordinate
* **long** - Longitude coordinate
* **sqft_living15** - The square footage of interior housing living space for the nearest 15 neighbors
* **sqft_lot15** - The square footage of the land lots of the nearest 15 neighbors
D

## Data Cleaning

### Dropping Uncessary Columns

In [ ]:
#df = df.drop(df[['id', 'date', 'view', 'lat', 'long', 'yr_renovated', 'sqft_living15', 'sqft_lot15']], axis=1)
#df.head()

In [ ]:
#seeing what the model looks like if I leave in sqft_living15 and sqft_lot15
df = df.drop(df[['id', 'date', 'view', 'lat', 'long', 'yr_renovated']], axis=1)
df.head()

### Checking Data Types

In [ ]:
df.info()

#### Removing the ? in sqft_basement

In [ ]:
basement_size = df['sqft_basement'].value_counts()
basement_size

In [ ]:
unknown_basements = df[df['sqft_basement'] == '?']
known_basements = df[df['sqft_basement'] != '?']

print(len(unknown_basements))
print(len(known_basements))

In [ ]:
sqft_basement = unknown_basements.apply(lambda x: x['sqft_living'] - x['sqft_above'], axis=1)
unknown_basements['sqft_basement'] = sqft_basement
unknown_basements.head()

In [ ]:
cleaned_df = known_basements.append(unknown_basements)
cleaned_df.head()

In [ ]:
#changing to float so that decminals are in the same format
cleaned_df['sqft_basement'] = cleaned_df['sqft_basement'].astype(float)
cleaned_df['sqft_above'] = cleaned_df['sqft_above'].astype(float)

In [ ]:
cleaned_df['sqft_basement'].value_counts().head()

#### Changing Zip Code to Category

In [ ]:
print(len(df['zipcode'].value_counts()))

In [ ]:
cleaned_df['zipcode'] = df['zipcode'].astype(str)

In [ ]:
cleaned_df['zipcode'].value_counts().head()

### Dropping Bedroom Outliers

In [ ]:
cleaned_df['bedrooms'].value_counts()

In [ ]:
#dropping outliers
cleaned_df = cleaned_df.sort_values('bedrooms', ascending=False).reset_index()
cleaned_df = cleaned_df.drop([0,1,2,3,4])
cleaned_df.head(5)

In [ ]:
cleaned_df.info()

### Exploring Data with Scatter Plot

In [ ]:
#using scatter plot to look for linear relationships
pd.plotting.scatter_matrix(cleaned_df, figsize = [20,20]);
plt.show()

### Analysis:
At first glance, the following variables seem to have linear relationships:
- price with bedrooms, sqft_above, & sqft_basement.
    - price also seems to have a linear relationship with categorical variable 'grade'.
- bedrooms with bathrooms, sqft_living, sqft_above, & sqft_basement
- sqft_living and sqft_above have the closest linear relationship
    - They are very similar data points. I may need to eliminate one to prevent multicolinearity.
    
The Following Variables seem to be categorical:
- floors
- waterfront
- condition
- zip code (not shown because I have already made it an object)

Ordinal Variables:
- bedrooms
- bathrooms



### What To Do with the Ordinal Values

I am going to treat grade as a continuous variable as it has very linear relationships with many features. Including price.

## Checking for Null Values

In [ ]:
cleaned_df.isna().sum()

In [ ]:
cleaned_df.info()

## Fixing Waterfront

In [ ]:
cleaned_df['waterfront'].value_counts()

In [ ]:
waterfront_cleaned = cleaned_df['waterfront'].fillna(0) 

In [ ]:
cleaned_df['waterfront'] = waterfront_cleaned

In [ ]:
cleaned_df['waterfront'].value_counts()

In [ ]:
cleaned_df.isna().sum()

# Test Model with no scaling/etc

In [ ]:
test_df = cleaned_df
#test_df.head()

In [ ]:
#df = df.reset_index()
#df = df.rename(columns={"index":"new_id"})
#df

In [ ]:
X = test_df.drop(['price', 'zipcode'], axis=1)
y = test_df['price']

In [ ]:
X_int = sm.add_constant(X)
model = sm.OLS(y,X_int).fit()
model.summary()

## Test Normalization

In [ ]:
def norm_feat(series):
    return (series - series.mean())/series.std()

In [ ]:
data= test_df
data.head()

In [ ]:
data.info()

In [ ]:
data = data.drop('zipcode', axis=1)

In [ ]:
df_norm = norm_feat(data)
df_norm.head()

## Test Model with Test Normalized Data

In [ ]:
X = df_norm.drop('price', axis=1)
y = df_norm['price']

In [ ]:
X_int = sm.add_constant(X)
model = sm.OLS(y,X_int).fit()
model.summary()

# Data Exploration

## Building Condition Explaination
https://info.kingcounty.gov/assessor/esales/Glossary.aspx?type=r#d (accessed 12/6/2021)

Relative to age and grade. Coded 1-5.

1 = Poor- Worn out. Repair and overhaul needed on painted surfaces, roofing, plumbing, heating and numerous functional inadequacies. Excessive deferred maintenance and abuse, limited value-in-use, approaching abandonment or major reconstruction; reuse or change in occupancy is imminent. Effective age is near the end of the scale regardless of the actual chronological age.

2 = Fair- Badly worn. Much repair needed. Many items need refinishing or overhauling, deferred maintenance obvious, inadequate building utility and systems all shortening the life expectancy and increasing the effective age.

3 = Average- Some evidence of deferred maintenance and normal obsolescence with age in that a few minor repairs are needed, along with some refinishing. All major components still functional and contributing toward an extended life expectancy. Effective age and utility is standard for like properties of its class and usage.

4 = Good- No obvious maintenance required but neither is everything new. Appearance and utility are above the standard and the overall effective age will be lower than the typical property.

5= Very Good- All items well maintained, many having been overhauled and repaired as they have shown signs of wear, increasing the life expectancy and lowering the effective age with little deterioration or obsolescence evident with a high degree of utility.


In [ ]:
cleaned_df['condition'].value_counts()

## Building Grade Explaination
https://info.kingcounty.gov/assessor/esales/Glossary.aspx?type=r#d (accessed 12/6/2021)


Represents the construction quality of improvements. Grades run from grade 1 to 13. Generally defined as:

1-3 Falls short of minimum building standards. Normally cabin or inferior structure.

4 Generally older, low quality construction. Does not meet code.

5 Low construction costs and workmanship. Small, simple design.

6 Lowest grade currently meeting building code. Low quality materials and simple designs.

7 Average grade of construction and design. Commonly seen in plats and older sub-divisions.

8 Just above average in construction and design. Usually better materials in both the exterior and interior finish work.

9 Better architectural design with extra interior and exterior design and quality.

10 Homes of this quality generally have high quality features. Finish work is better and more design quality is seen in the floor plans. Generally have a larger square footage.

11 Custom design and higher quality finish work with added amenities of solid woods, bathroom fixtures and more luxurious options.

12 Custom design and excellent builders. All materials are of the highest quality and all conveniences are present.

13 Generally custom designed and built. Mansion level. Large amount of highest quality cabinet work, wood trim, marble, entry ways etc.

## Should I Drop Low Grade Houses?

In [ ]:
cleaned_df['grade'].value_counts()

I am only concerned with houses that currently meet the building code, so I will drop the houses which are grade 5 or lower

In [ ]:
low_grade = cleaned_df[cleaned_df['grade'] <= 5]
low_grade.head()

In [ ]:
low_grade['condition'].value_counts()

In [ ]:
low_grade['waterfront'].value_counts()

## Checking for Multicolinearity

In [ ]:
cleaned_df.head(1)

In [ ]:
#numeric variables
numeric_vars= ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront',
'condition', 'grade', 'sqft_above', 'sqft_basement', 'sqft_living15', 'sqft_lot15']
numeric_vars

In [ ]:
king_preprocessed = cleaned_df.loc[:,numeric_vars]
king_preprocessed.head()

In [ ]:
#scatter matrix for numeric variables
pd.plotting.scatter_matrix(king_preprocessed, figsize=[20, 20]);

In [ ]:
#correlation matrix
king_preprocessed.corr()

In [ ]:
abs(king_preprocessed.corr()) > 0.75

In [ ]:
cc_df = king_preprocessed.corr().abs().stack().reset_index().sort_values(0, ascending=False)

cc_df['pairs'] = list(zip(cc_df.level_0, cc_df.level_1))

cc_df.set_index(['pairs'], inplace = True)

cc_df.drop(columns=['level_1', 'level_0'], inplace = True)

# cc for correlation coefficient
cc_df.columns = ['cc']

cc_df.drop_duplicates(inplace=True)

cc_df[(cc_df.cc>.75) & (cc_df.cc<1)]

I need to remove some of these variables to prevent multicolineartiy. 


### Multicollinearity Analysis

There are 4 sets of variables that are highly correlated.  

In [ ]:
#removing these two variables removes all of the colinearity pairs
#king_preprocessed.drop(columns=['sqft_above', 'sqft_living'], inplace=True)

In [ ]:
king_preprocessed.drop(columns=['sqft_above'], inplace=True)

In [ ]:
#checking for correlated pairs again
cc_df = king_preprocessed.corr().abs().stack().reset_index().sort_values(0, ascending=False)

cc_df['pairs'] = list(zip(cc_df.level_0, cc_df.level_1))

cc_df.set_index(['pairs'], inplace = True)

cc_df.drop(columns=['level_1', 'level_0'], inplace = True)

# cc for correlation coefficient
cc_df.columns = ['cc']

cc_df.drop_duplicates(inplace=True)

cc_df[(cc_df.cc>.75) & (cc_df.cc<1)]

In [ ]:
king_preprocessed

## Check VIF Scores!!!!

## Normalizing Data

In [ ]:
def norm_feat(series):
    return (series - series.mean())/series.std()

In [ ]:
data = norm_feat(king_preprocessed)
data

In [ ]:
df_norm = norm_feat(data)
df_norm

In [ ]:
sns.pairplot(df_norm)

In [ ]:
#sns.heatmap(df_norm);

In [ ]:
#data_pred = df_norm

## Plotting Categoricals and then One-Hot Encoding them

## Dealing with Zip Code
I Need to get Zipcode back in the dataframe so that I can encode it.
Maybe I should do the One Hot Encoding much earlier in my workflow.

In [ ]:
#categoricals = df_norm[['bedrooms', 'bathrooms', 'floors','condition', 'waterfront', 'grade']]

In [ ]:
#categoricals = df_norm[['floors','waterfront', 'grade']]

In [ ]:
#categoricals.head()

In [ ]:
#king_preprocessed

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(20,10), sharey=True)

for col, ax in zip(categoricals, axes.flatten()):
    (king_preprocessed.groupby(col)               # group values together by column of interest
         .mean()['price']        # take the mean of the saleprice for each group
         .sort_values()              # sort the groups in ascending order
         .plot
         .bar(ax=ax))                # create a bar graph on the ax
    
    ax.set_title(col)                # Make the title the name of the column
    
fig.tight_layout()


In [ ]:
#sns.heatmap(categoricals)

In [ ]:
#correlation matrix. 
#df.corr()

In [ ]:
#df_pred_222 = abs(df.corr() > 0.75)

### One Hot Encoding

In [ ]:
feats = ['floors', 'waterfront', 'condition']
feats_df = king_preprocessed[feats].astype(str)
encoded_df = pd.get_dummies(feats_df, drop_first=True)
encoded_df.head()

In [ ]:
df = pd.get_dummies(feats_df, drop_first=True)
df

In [ ]:
df

In [ ]:
pd.get_dummies(feats_df, )

In [ ]:
pd.get_dummies(king_preprocessed.grade)

In [ ]:
#trying sklearn method
ohe = OneHotEncoder(drop='first')

In [ ]:
king_preprocessed.head(1)

In [ ]:
data_trans = ohe.fit_transform(king_preprocessed)
data_trans.todense()

In [ ]:
data_trans = pd.DataFrame(data_trans.todense(), columns=ohe.get_feature_names())
data_trans.head()

# Modeling

# Archive

In [ ]:
#df['waterfront'].value_counts()

In [ ]:
#df['waterfront'] = df['waterfront'].astype('bool') #converting waterfront into boolean

## Resetting IDs to make them shorter

In [ ]:
X = test_df.drop('price', axis=1)
y = test_df['price']